# Imports

In [1]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict

# Années précédentes

In [45]:
years=[i for i in range(2011,2025)]
years

[2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]

In [46]:
url_list=[f'https://fr.wikipedia.org/wiki/Miss_France_{year}' for year in years]
url_list

['https://fr.wikipedia.org/wiki/Miss_France_2011',
 'https://fr.wikipedia.org/wiki/Miss_France_2012',
 'https://fr.wikipedia.org/wiki/Miss_France_2013',
 'https://fr.wikipedia.org/wiki/Miss_France_2014',
 'https://fr.wikipedia.org/wiki/Miss_France_2015',
 'https://fr.wikipedia.org/wiki/Miss_France_2016',
 'https://fr.wikipedia.org/wiki/Miss_France_2017',
 'https://fr.wikipedia.org/wiki/Miss_France_2018',
 'https://fr.wikipedia.org/wiki/Miss_France_2019',
 'https://fr.wikipedia.org/wiki/Miss_France_2020',
 'https://fr.wikipedia.org/wiki/Miss_France_2021',
 'https://fr.wikipedia.org/wiki/Miss_France_2022',
 'https://fr.wikipedia.org/wiki/Miss_France_2023',
 'https://fr.wikipedia.org/wiki/Miss_France_2024']

In [47]:
# Dictionnaire pour stocker les DataFrames
dataframes = {}

In [48]:
def extract_candidates_table(url, year):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Localiser la section "Candidates"
    section_candidates = None
    for header in soup.find_all('h2'):
        if "Candidates" in header.text:
            section_candidates = header
            break
    
    if section_candidates:
        # Trouver la table qui suit immédiatement cette section
        candidates_table = section_candidates.find_next('table', {'class': 'wikitable'})
        if candidates_table:
            # Extraire les en-têtes de la table
            headers = [header.text.strip() for header in candidates_table.find_all('th')]
            # Extraire les lignes de la table
            rows = []
            for row in candidates_table.find_all('tr')[1:]:  # Ignorer les en-têtes
                cols = [col.text.strip() for col in row.find_all('td')]
                rows.append(cols)
            # Créer un DataFrame pandas
            df = pd.DataFrame(rows, columns=headers)
            df['année'] = year
            return df
    return None

In [49]:
for url in url_list:
    year = url.split('_')[-1]  # Extraire l'année de l'URL
    df = extract_candidates_table(url, year)
    if df is not None:
        dataframes[f"année {year}"] = df
        print(f"Données extraites pour l'année {year}.")
    else:
        print(f"Pas de table 'Candidates' trouvée pour l'année {year}.")

Données extraites pour l'année 2011.
Données extraites pour l'année 2012.
Données extraites pour l'année 2013.
Données extraites pour l'année 2014.
Données extraites pour l'année 2015.
Données extraites pour l'année 2016.
Données extraites pour l'année 2017.
Données extraites pour l'année 2018.
Données extraites pour l'année 2019.
Données extraites pour l'année 2020.
Données extraites pour l'année 2021.
Données extraites pour l'année 2022.
Données extraites pour l'année 2023.
Données extraites pour l'année 2024.


In [50]:
print(dataframes.keys())

dict_keys(['année 2011', 'année 2012', 'année 2013', 'année 2014', 'année 2015', 'année 2016', 'année 2017', 'année 2018', 'année 2019', 'année 2020', 'année 2021', 'année 2022', 'année 2023', 'année 2024'])


In [51]:
categories = defaultdict(list)

# Identifier les headers de chaque DataFrame et les regrouper
for year, df in dataframes.items():
    headers = tuple(df.columns)  # Les colonnes de chaque DataFrame sous forme de tuple
    categories[headers].append(year)

# Afficher les catégories et les années associées
for idx, (header_set, years) in enumerate(categories.items(), start=1):
    print(f"Catégorie {idx}:")
    print("Headers :", header_set)
    print("Années :", years)
    print("-" * 50)


Catégorie 1:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Classement final', 'année')
Années : ['année 2011', 'année 2014', 'année 2015', 'année 2016', 'année 2021']
--------------------------------------------------
Catégorie 2:
Headers : ('Région', 'Nom[2]', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Classement final', 'année')
Années : ['année 2012', 'année 2013']
--------------------------------------------------
Catégorie 3:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Résultat', 'année')
Années : ['année 2017']
--------------------------------------------------
Catégorie 4:
Headers : ('région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Groupe', 'Classement final', 'année')
Années : ['année 2018']
--------------------------------------------------
Catégorie 5:
Headers : ('région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Classement final', 'année')
Années : ['année 2019']
----------------------------------------------

# Nettoyage des colonnes

In [58]:
def clean_columns(df):
    df.columns = df.columns.str.replace(r'\[.*?\]', '', regex=True).str.strip()
    df.columns = df.columns.str.replace(r'^région$', 'Région', regex=True)
    df.columns = df.columns.str.replace(r'Élue le','Date élection', regex=True)
    df.columns = df.columns.str.replace(r'Élection régionale','Date élection', regex=True)
    df.columns = df.columns.str.replace(r'final$','Final', regex=True)
    df.columns = df.columns.str.replace(r'Résultat','Classement Final', regex=True)
    if 'Réf.' in df.columns:
        df = df.drop(columns=['Réf.'])
    if 'Groupe' in df.columns:
        df = df.drop(columns=['Groupe'])
    return df

for year, df in dataframes.items():
    dataframes[year] = clean_columns(df)

for year, df in dataframes.items():
    print(f"Année : {year}")
    print(df.columns)
    print("-" * 50)

Année : année 2011
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2012
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2013
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2014
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2015
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------


In [59]:
categories = defaultdict(list)

for year, df in dataframes.items():
    headers = tuple(df.columns)  
    categories[headers].append(year)

for idx, (header_set, years) in enumerate(categories.items(), start=1):
    print(f"Catégorie {idx}:")
    print("Headers :", header_set)
    print("Années :", years)
    print("-" * 50)

Catégorie 1:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection', 'Classement Final', 'année')
Années : ['année 2011', 'année 2012', 'année 2013', 'année 2014', 'année 2015', 'année 2016', 'année 2017', 'année 2018', 'année 2019', 'année 2020', 'année 2021', 'année 2022', 'année 2023', 'année 2024']
--------------------------------------------------


In [60]:
df_miss = pd.concat(dataframes.values(), ignore_index=True)

print("DataFrame fusionnée :")
print(df_miss.head())
print(f"Taille de la DataFrame fusionnée : {df_miss.shape}")

DataFrame fusionnée :
           Région                 Nom     Âge  Taille         Résidence  \
0     Miss Alsace    Mathilde Buecher  19 ans  1,78 m       Heimersdorf   
1  Miss Aquitaine      Clémence Thill  20 ans  1,75 m              Agen   
2   Miss Auvergne     Clémence Oleksy  19 ans  1,76 m             Vichy   
3  Miss Bourgogne  Alice Detollenaere  23 ans  1,74 m  Chitry-les-Mines   
4   Miss Bretagne     Laury Thilleman  19 ans  1,78 m             Brest   

                         Date élection  \
0          10 octobre 2010 à Hirsingue   
1           16 octobre 2010 à Cadillac   
2            17 octobre 2010 à Cébazat   
3  1er octobre 2010 à Chalon-sur-Saône   
4            26 octobre 2010 à Pontivy   

                                Classement Final année  
0                                                 2011  
1                                                 2011  
2                                    2e dauphine  2011  
3                            Prix du Mannequin

# Transformation des données

## Données catégoriques

In [61]:
categorical_columns = ['Région']

for col in categorical_columns:
    unique_values = df_miss[col].unique()
    print(f"Valeurs uniques dans la colonne '{col}':")
    print(unique_values)
    print("-" * 50)

Valeurs uniques dans la colonne 'Région':
['Miss Alsace' 'Miss Aquitaine' 'Miss Auvergne' 'Miss Bourgogne'
 'Miss Bretagne' 'Miss Centre' 'Miss Champagne-Ardenne' 'Miss Corse'
 "Miss Côte d'Azur" 'Miss Franche-Comté' 'Miss Guadeloupe' 'Miss Guyane'
 'Miss Île-de-France' 'Miss Languedoc' 'Miss Limousin' 'Miss Lorraine'
 'Miss Martinique' 'Miss Mayotte' 'Miss Midi-Pyrénées'
 'Miss Nord-Pas-de-Calais' 'Miss Normandie' 'Miss Nouvelle-Calédonie'
 'Miss Orléanais' 'Miss Pays de Loire' 'Miss Pays de Savoie'
 'Miss Picardie' 'Miss Poitou-Charentes' 'Miss Provence' 'Miss Réunion'
 'Miss Rhône-Alpes' 'Miss Roussillon' 'Miss Saint-Pierre-et-Miquelon'
 'Miss Tahiti' 'Miss Saint-Martin' 'Miss Centre-Val-de-Loire'
 'Miss Guadeloupe et Îles du Nord' 'Miss Centre-Val de Loire'
 'Miss Languedoc-Roussillon' 'Miss Saint-Martin et Saint-Barthélemy'
 'Miss Pays de la Loire' 'Miss Wallis-et-Futuna'
 'Miss Saint-Martin et Saint-Barthélémy']
--------------------------------------------------


In [62]:
cleaning_dict = {
    "Miss Saint-Martin et Saint-Barthélémy": "Miss Saint-Martin et Saint-Barthélemy",
    "Miss Saint-Martin et Saint-Barthélemy": "Miss Saint-Martin et Saint-Barthélemy",
    "Miss Centre-Val de Loire": "Miss Centre-Val-de-Loire",
    "Miss Pays de Loire": "Miss Pays de la Loire",
    "Miss Guadeloupe et Îles du Nord": "Miss Guadeloupe",
    "Miss Wallis-et-Futuna": "Miss Wallis et Futuna",
    "Miss Saint-Martin": "Miss Saint-Martin et Saint-Barthélemy",
}

df_miss['Région'] = df_miss['Région'].replace(cleaning_dict)

unique_regions = df_miss['Région'].unique()
print("Valeurs uniques après nettoyage :")
print(unique_regions)

pd.DataFrame({"Région": unique_regions}).to_csv("regions_cleaned.csv", index=False)

Valeurs uniques après nettoyage :
['Miss Alsace' 'Miss Aquitaine' 'Miss Auvergne' 'Miss Bourgogne'
 'Miss Bretagne' 'Miss Centre' 'Miss Champagne-Ardenne' 'Miss Corse'
 "Miss Côte d'Azur" 'Miss Franche-Comté' 'Miss Guadeloupe' 'Miss Guyane'
 'Miss Île-de-France' 'Miss Languedoc' 'Miss Limousin' 'Miss Lorraine'
 'Miss Martinique' 'Miss Mayotte' 'Miss Midi-Pyrénées'
 'Miss Nord-Pas-de-Calais' 'Miss Normandie' 'Miss Nouvelle-Calédonie'
 'Miss Orléanais' 'Miss Pays de la Loire' 'Miss Pays de Savoie'
 'Miss Picardie' 'Miss Poitou-Charentes' 'Miss Provence' 'Miss Réunion'
 'Miss Rhône-Alpes' 'Miss Roussillon' 'Miss Saint-Pierre-et-Miquelon'
 'Miss Tahiti' 'Miss Saint-Martin et Saint-Barthélemy'
 'Miss Centre-Val-de-Loire' 'Miss Languedoc-Roussillon'
 'Miss Wallis et Futuna']
